**GEO6361 (semaine 8) : Les opérations spatiales avec GeoPandas**

Cette semaine, nous allons nous intéresser à quelques opérations fondamentales des SIG qu'il nous est possible d'effectuer avec GeoPandas.

## **1. Opérations spatiales avec GeoPandas**

### **1.1 Installons les modules requis pour cette section, et importons-les**

In [ ]:
import pandas as pd # On importe pandas, on lui attribut l'alias pd
import geopandas as gpd # On importe GeoPandas et on lui attribut l'alias gpd

### **1.2 Création d'objets spatiaux avec GeoPandas**

**Commençons par créer les point de ces stations** (pour ça, on importe également quelques outils de création d'objets spatiaux d'un autre module externe : Shapely (https://github.com/shapely/shapely), qui est fortement utilisé par GeoPandas pour représenter ses entités spatiales)

In [ ]:
from shapely.geometry import Polygon, LineString, Point # On peut importer différents types de géométries, même si nous n'utiliserons que "Point" aujourd'hui

Les objets spatiaux Shapely peuvent être créés à partir de liste de coordonnées :

In [ ]:
coord1 = [-73.62367, 45.52339]
coord2 = [-73.61516, 45.52014]

pt1 = Point(coord1) # Créer un point à partir du premier jeu de coordonnées
pt2 = Point(coord2) # Créer un point à partir du deuxième jeu de coordonnées

print(pt1)
print(pt2)

In [ ]:
# Ces points ne sont pas encore intégrés à une GeoDataFrame... Faisons-le afin de pouvoir les manipuler avec les fonctions et méthodes GeoPandas :
description = {'nom': ['Station Acadie', 'Station Outremont'], 'geometry': [pt1, pt2]} # Création d'un dictionnaire Python

# Intégrons ces données dans une GeoDataFrame...
points = gpd.GeoDataFrame(description, geometry='geometry')

In [ ]:
# Quel est le système de projection ?
print(points.crs)

In [ ]:
# Attribuons-lui son système de projection...
points.crs = 'EPSG:4326'

In [ ]:
# ...et vérifions :
print(points.crs)
points.crs

In [ ]:
# De quoi notre GeoDataFrame a-t-elle l'air ?
points

In [ ]:
# De quoi ce GDF a-t-il l'air ?
points.plot()

**Quelle distance sépare ces deux points ?** Utilisons la méthode "[distance](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoSeries.distance.html)" de GeoPandas :

In [ ]:
# Par curiosité, quelle est la distance entre les deux points ?
points.iloc[0]['geometry'].distance(points.iloc[1]['geometry']) # prendre une entité et lui appliquer la méthode "distance" à laquelle on passe une autre entité

La méthode "distance" fournie un résultat dans l'unité du système de coordonnées des points. Les coordonnées géographiques étant exprimées en angles, la distance est exprimée en degrées. **Si l'on souhaite des résultats en mètres, nous devons au préalable reprojeter nos données avec "[to_crs](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.to_crs.html)"** :

In [ ]:
# Reprojetons, par exemple vers EPSG:32188 (MTM Zone 8) :
points = points.to_crs('epsg:32188')

# Puis recalculons la distance :
points.iloc[0]['geometry'].distance(points.iloc[1]['geometry'])

### **1.3 Création d'une zone tampon ("buffer") de 500m autour de ces deux points**

**On peut ensuite produire une géométrie du "[buffer](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoSeries.buffer.html)" de la manière suivante :**

In [ ]:
# Créons une zone tampons de 500 mètres autour de ces deux points...
buffered_points = points.buffer(500, resolution=16)
buffered_points

In [ ]:
# ... et visualisons-la
buffered_points.plot(alpha=.5, figsize=(6,6))

### **1.4 Quelle est la superficie de la zone totale ?**

**Nous pouvons agréger ces polygones avec une union géométrique ("[union_all](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoSeries.union_all.html)")**

In [ ]:
# Créer la géométrie correspondant à l'aggregation des polygones du GeoDataFrame en utilisant la méthode union_all()
buffered_points_union = buffered_points.union_all()
print(type(buffered_points_union)) # On vérifie qu'on obtient bien une géométrie

# Créer une GeoDataFrame à partir de cette géométrie (pour pouvoir bénéficier des méthodes de GeoPandas)
buffered_points_union_gdf = gpd.GeoDataFrame(geometry=[buffered_points_union])

# Renseigner son CRS
print(buffered_points_union_gdf.crs)
buffered_points_union_gdf.crs = 'EPSG:32188'
print(buffered_points_union_gdf.crs)

In [ ]:
# Visualisons la GeoDataFrame :
buffered_points_union_gdf.plot(alpha=.5, figsize=(5,5))

**Et obtenir la superficie et le pémimètre en appelant "[area](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoSeries.area.html)" puis "[length](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoSeries.length.html)" sur cet objet :**

In [ ]:
# Calculons la superficie :
print(f"La superficie de la géométrie des deux buffers est de {int(buffered_points_union_gdf.area[0])} mètres carrés")

# Calculons le périmètre :
print(f"Le périmètre de la géométrie des deux buffers est de {int(buffered_points_union_gdf.length[0])} mètres")

In [ ]:
# On peut aussi exporter cet objet pour l'utiliser dans une autre application :
buffered_points_union_gdf.to_file("/content/buffers_union.geojson", driver='GeoJSON')

### **1.5 Quelle est la superficie de la zone située à la fois à moins de 500m de la station Acadie, et à moins de 500m de la station Outremont ?**

**Nous pouvons utiliser la fonction "[intersection](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoSeries.intersection.html)" sur la GeoDataFrame où les tampons sont séparés (c'est à dire, pas sur la version "union")**

In [ ]:
# Créer la géométrie correspondant à l'intersection des polygones du GeoDataFrame en utilisant la méthode intersect()
buffered_points_intersect = buffered_points[0].intersection(buffered_points[1])

# Calculons la superficie de cette géométrie
surf = buffered_points_intersect.area
print(f"La superficie de l'intersection des deux buffers est de {int(surf)} mètres carrés")

In [ ]:
# Créer une GeoDataFrame à partir de cette géométrie (pour pouvoir bénéficier des méthodes de GeoPandas)
buffered_points_intersect_gdf = gpd.GeoDataFrame(geometry=[buffered_points_intersect])
buffered_points_intersect_gdf.crs = 'EPSG:32188'

buffered_points_intersect_gdf.plot()

### **1.6 Quelle est la superficie des deux buffers à l'exeption de leur zone commune ?**

In [ ]:
buffered_points[0].difference(buffered_points[1]) # prendre une entité et lui appliquer la méthode "différence" à laquelle on passe une autre entité

In [ ]:
buffered_points[1].difference(buffered_points[0])

In [ ]:
surf = buffered_points[0].difference(buffered_points[1]).area + buffered_points[1].difference(buffered_points[0]).area
print(f"La superficie des deux buffers, à l'exeption de leur zone commune, est de {int(surf)} mètres carrés")

In [ ]:
# On peut le faire plus rapidement avec "symmetric_difference" : https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoSeries.symmetric_difference.html#geopandas.GeoSeries.symmetric_difference
buffered_points[0].symmetric_difference(buffered_points[1]).area

## **2. Une application un peu plus poussée**

Nous avons vu une application simple des opérations de "**buffer**", d'**union**, d'**intersection**, et de **différence**, et des calculs de **superficie** et de **périmètres**.
Allons plus loin avec une application plus poussée : essayons de calculer des statistiques sur une base de données en fonction de distances par rapport aux éléments d'une autre base de données.

Plus précisément, nous allons prendre une base de données spatiale de prix de biens immobiliers, et explorer le rapport entre ces prix et leur distance aux stations de métro de Montréal.

### **1.2 Chargeons les données**

 Chargeons tout d'abord les données sur les biens immobiliers en vente pour 2023 :

In [ ]:
# On charge le fichier contenant les données spatiales à analyser dans une GDF
immo2023 = gpd.read_file('/content/data_immo_quebec_2023.geojson')
immo2023.crs = 'EPSG:4326'

# On reprojette la GDF vers EPSG:32188 (MTM Zone 8), on pourrait évidemment utiliser d'autres systèmes de projection (ex: EPSG:3798 pour Lambert MTQ)
immo2023 = immo2023.to_crs('epsg:32188')

In [ ]:
# Cartographions rapidement ces données
immo2023.plot(figsize=(10,10), markersize=.5)

**Comme nous ne nous interressons qu'aux données de l'île de Montréal, excluons les points qui sont situés à l'extérieur. Pour accomplir cela, on peut numériser le contour de l'île avec QGIS ou geojson.io** (un fichier masque_mtl.geojson a été préparé pour ça), et "clipper" les points par rapport à celui-ci.

In [ ]:
# Charger un fichier GeoJSON contenant un masque de l'île de Montréal
masque = gpd.read_file('/content/masque_mtl.geojson')
masque.crs = 'EPSG:4326'

# On reprojette le GDF vers EPSG:32188 (MTM Zone 8)
masque = masque.to_crs('epsg:32188')
masque.plot()

In [ ]:
# On découpe (ou "clippe") les données avec le masque de Montréal
immo2023 = immo2023.clip(masque) # on passe le masque de découpage à la méthode clip appliquée à la GDF des données à découper.

In [ ]:
immo2023.plot()

**Lisons maintenant la carte des stations de métro de la STM**

In [ ]:
# Charger un fichier GeoJSON contenant les stations de métro de Montréal (https://www.stm.info/fr/a-propos/developpeurs)
stm = gpd.read_file('/content/stm_metro.geojson')
stm.crs = 'EPSG:4326'

# On reprojette le GDF vers EPSG:32188 (MTM Zone 8)
stm = stm.to_crs('epsg:32188')

In [ ]:
# Cartographions rapidement ces données :
stm.plot(figsize=(10,10),markersize=2)

In [ ]:
# Créer un buffer de 500 mètres autour des stations de métro
buffered_stm = stm.buffer(500)

In [ ]:
# Créer la géométrie correspondant à l'agregation des polygones du GeoDataFrame
buffered_stm = buffered_stm.union_all()

# Créer une GeoDataFrame à partir de cette géométrie
buffered_stm = gpd.GeoDataFrame(crs=stm.crs, geometry=[buffered_stm])

In [ ]:
# Cartographier le buffer agrégé :
buffered_stm.plot(figsize=(10,10), alpha=.5)

### **2.3. Faire une jointure spatiale pour calculer les prix à l'intérieur des buffers**

**Nous voudrions sélectionner les biens immobiliers qui sont géographiquement localisés à l'intérieur d'une géométrie.**

In [ ]:
# Effectuer la jointure spatiale
buffered_stm_immo = immo2023.sjoin(buffered_stm, how="inner", predicate='within')

In [ ]:
# Visualisons le résultat
buffered_stm_immo.plot(figsize=(10,10), markersize=.5)

In [ ]:
# On peut manipuler nos données comme n'importe quelle DataFrame
buffered_stm_immo.info()

In [ ]:
# Pour retirer des colonnes inutiles
buffered_stm_immo = buffered_stm_immo.drop(['BedRoom', 'FullAddress', 'lat', 'lon', 'index_right'], axis=1)

In [ ]:
# Si on ne veut garder que les annonces dont les prix sont suppérieurs à 40000$
buffered_stm_immo = buffered_stm_immo[
    buffered_stm_immo['Prix'] > 40000
    ]

In [ ]:
# Calculer des statistiques
print(buffered_stm_immo['Prix'].mean())
print(buffered_stm_immo['Prix'].median())

In [ ]:
print(f"Le prix médian des biens immobiliers situés à moins de 500m d'une station de métro est de {round(buffered_stm_immo['Prix'].median())}$")
print(f"Le prix médian des biens immobiliers est de {round(immo2023['Prix'].median())}$")

### **2.4. Et si nous voulions automatiser tout ça ?**

In [ ]:
import seaborn as sns # On importe le module de visualisation Seaborn...
import numpy as np # Et Numpy qui nous sera également utile

In [ ]:
distances = range(100,2100,250) # on crée une liste de distances de buffers à travers lesquels calculer nos statistiques

prix_moyens = []

for dist in distances:
    print(f"Buffer de {dist} mètres autour des stations de métro.")
    stmB = stm.buffer(dist)
    stmB = stmB.union_all()
    stmB = gpd.GeoDataFrame(crs=stm.crs, geometry=[stmB])

    prix_moyens.append(immo2023.sjoin(stmB, how="inner", predicate='within')['Prix'].mean())

data = np.array([distances, prix_moyens])
data = np.transpose(data)
prix_moyens_dist = pd.DataFrame(data, columns=['distances', 'prix_moyens'])

sns.scatterplot(data=prix_moyens_dist, x='distances', y='prix_moyens')